In [ ]:
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
import time
import json
import requests
from requests.exceptions import SSLError
import psycopg2
import random
import csv
from email.mime.text import MIMEText
from email.utils import COMMASPACE
import smtplib

headers = {
    'authority': 'moto.av.by',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "YaBrowser";v="24.1", "Yowser";v="2.5"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36',
}
    # если теперь что-то будет не так, то попробовать requests.Session()
    #подставить ключи если будет надо
    #"X-Api-Key": "-",
    #"X-User-Group": "-",

# Чтение json конфига
with open("config.json") as file:
    config = json.load(file)

mail_login = config['sender login']
mail_password = config['sender password']
pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']
recipients = config['mail recipients']

# Чтение json исключений
with open('exceptions.json', encoding="utf8") as file:
    exceptions_json = json.load(file)

exclude_sellers = exceptions_json['exclude_sellers']
exclude_brands = exceptions_json['exclude_brands']

#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)

current_time_start = datetime.now()
ctsf = current_time_start.strftime("%Y-%m-%d %H:%M:%S")
print(f"Привет! Текущая дата - {ctsf}")

cursor = conn.cursor()
# Посчитываем кол-во строк
select_query = "SELECT count(*) from av_full"
cursor.execute(select_query)
rows_full = cursor.fetchone()[0]
print(f"Строк в базе: {rows_full}")

Привет! Текущая дата - 2025-04-03 13:26:49
Строк в базе: 51396


In [9]:
cursor.execute('select id, seller_ph_nr from av_full where id = 114579056')
id_row = cursor.fetchone()
id_value = id_row[0]
seller_ph_nr = id_row[1]

In [ ]:
# Функция проверки на дубликаты
def check_for_duplicates (id_value, seller_ph_nr):
    # Ищем дубликаты по id_value и номеру телефона
    select_query = """SELECT id, date, price, model_vlk, brand, model, model_misc, year, type, cylinders, capacity, mileage, duplicate_flag, url, seller_ph_nr
    FROM public.av_full
    WHERE (brand, model, year, type, cylinders, capacity, locations) = 
    (
        SELECT brand, model, year, type, cylinders, capacity, locations
        FROM public.av_full
        WHERE id = %d
    )
    AND seller_ph_nr && ARRAY%s
    OR id = %d
    ORDER BY date ASC""" % (id_value, seller_ph_nr, id_value)
    cursor.execute(select_query)
    dupl_rows = cursor.fetchall()
    dupl_count = 0 # счетчик количества объяв-дубликатов
    is_not_yet_marked_as_duplicate = 0 # были ли в выборке уже помеченные как дубликаты объявы

    if len(dupl_rows) > 1:
        dupl_id_list = []
        dupl_date_list = []

        for dupl_n in dupl_rows:    
            dupl_dates = dupl_n[1]
            dupl_id = dupl_n[0]
            dupl_date_list.append(dupl_dates)
            dupl_id_list.append(dupl_id)
            #print(f"id - {dupl_id}, дата - {dupl_dates}")
            if dupl_n[12] is False:
                is_not_yet_marked_as_duplicate += 1

        # Находим в списках самую раннюю дату и самый поздний айди
        dupl_date = min(dupl_date_list)
        dupl_main_id = max(dupl_id_list)
            
        # отформатировать список для вставки в квери
        dupl_id_list.remove(dupl_main_id)
        dupl_count = len(dupl_id_list)
        dupl_id_list = ', '.join(str(int(d)) for d in dupl_id_list)
        
        print(f"Найдено {dupl_count} дубликатов для id:{id_value} с самой ранней датой - {dupl_date}")

        # квери выставить флажок дубликата
        query1 = """UPDATE public.av_full
        SET duplicate_flag = True, duplicate_id = %d
        WHERE id in (%s);""" % (dupl_main_id, dupl_id_list)
        cursor.execute(query1)

        # квери выставить новую дату
        query2 = """UPDATE public.av_full
        SET date_corrected = '%s'
        WHERE id = %d;""" % (dupl_date, dupl_main_id)
        cursor.execute(query2)
            
        # записать изменения
        conn.commit()
        if is_not_yet_marked_as_duplicate > 0:
            return 1
    else:
        print(f'Дубликатов для id:{id_value} не обнаружено')
        return 0

# далее взять айди с самой свежей датой (или самый большой айди?) и вписать ему самую старую дату из списка
# исключить его из того списка? и оставшимся записать тот айди и выставить флаг


In [10]:
# Ищем дубликат по id_value, список не будет содержать строку искомого айди
select_query = """SELECT id, date, price, model_vlk, brand, model, model_misc, year, type, cylinders, capacity, mileage, duplicate_flag, url, seller_ph_nr
FROM public.av_full
WHERE (brand, model, year, type, cylinders, capacity, locations) = 
(
    SELECT brand, model, year, type, cylinders, capacity, locations
    FROM public.av_full
    WHERE id = %d
    
)
AND seller_ph_nr && ARRAY%s
OR id = %d
ORDER BY date ASC""" % (id_value, seller_ph_nr, id_value)


In [7]:
cursor.execute(select_query)
dupl_rows = cursor.fetchall()
dupl_count = 0 # счетчик количества объяв-дубликатов
is_not_yet_marked_as_duplicate = 0 # были ли в выборке уже помеченные как дубликаты объявы

if len(dupl_rows) > 1:
    dupl_id_list = []
    dupl_date_list = []

    for dupl_n in dupl_rows:    
        dupl_dates = dupl_n[1]
        dupl_id = dupl_n[0]
        dupl_date_list.append(dupl_dates)
        dupl_id_list.append(dupl_id)
        #print(f"id - {dupl_id}, дата - {dupl_dates}")
        if dupl_n[12] is False:
            is_not_yet_marked_as_duplicate += 1

    # Находим в списках самую раннюю дату и самый поздний айди
    dupl_date = min(dupl_date_list)
    dupl_main_id = max(dupl_id_list)
        
    # отформатировать список для вставки в квери
    dupl_id_list.remove(dupl_main_id)
    dupl_count = len(dupl_id_list)
    dupl_id_list = ', '.join(str(int(d)) for d in dupl_id_list)
    
    print(f"Найдено {dupl_count} дубликатов для id:{id_value} с самой ранней датой - {dupl_date}")

    # квери выставить флажок дубликата
    query1 = """UPDATE public.av_full
    SET duplicate_flag = True, duplicate_id = %d
    WHERE id in (%s);""" % (dupl_main_id, dupl_id_list)
    cursor.execute(query1)

    # квери выставить новую дату
    query2 = """UPDATE public.av_full
    SET date_corrected = '%s'
    WHERE id = %d;""" % (dupl_date, dupl_main_id)
    cursor.execute(query2)
    
    # записать изменения
    #                                                                      conn.commit()

Найдено 9 дубликатов для id:114579056 с самой ранней датой - 2025-02-26 18:54:32
